# Decision Trees

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Obtaining the data

In [2]:
data = pd.read_csv('../../Data/housing_clean.csv')
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3


In [3]:
label = 'median_house_value'

In [4]:
for feature in data.columns:
    if feature == label : continue

    data[feature] = (data[feature] - data[feature].min())/(data[feature].max() - data[feature].min())

In [5]:
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0.211155,0.567481,0.784314,0.022331,0.019863,0.008941,0.020556,0.539668,452600.0,0.75
1,0.212151,0.565356,0.392157,0.180503,0.171477,0.067210,0.186976,0.538027,358500.0,0.75
2,0.210159,0.564293,1.000000,0.037260,0.029330,0.013818,0.028943,0.466028,352100.0,0.75
3,0.209163,0.564293,1.000000,0.032352,0.036313,0.015555,0.035849,0.354699,341300.0,0.75
4,0.209163,0.564293,1.000000,0.041330,0.043296,0.015752,0.042427,0.230776,342200.0,0.75


In [6]:
train_data = data[:int(0.8*len(data))]
test_data = data[int(0.8*len(data)):]

In [7]:
train_data_X = train_data.drop(label,axis=1)
train_data_Y = train_data[label]
test_data_X = test_data.drop(label,axis=1)
test_data_Y = test_data[label]

## Defining the model

### Decision Tree 

A decision tree is a binary tree where a decision is made at each node based on a feature and a threshold. The sample points are filtered more and more the further down the tree one goes until ideally, the samples are all of the same class (for classification). Such nodes (called pure nodes) do not need to be divided further and become leaf nodes. For regression, the concept of pure nodes do not exist.

To prevent overfitting though, trees are usually not allowed to grow beyond a certain max_depth which usually results in impure leaf nodes. Here, one can take the mode of the samples for classification. For regression, one will always take the mean of any leaf node. 

The impurity of a node can be measure with Gini impurity for classification and Variance for regression. We want to minimize the gini impurity/variance for each node or in other words **maximize the decrease** in impurity from a parent node to child nodes.

$$\text{Gini impurity, }G = 1 - \sum_k p_k^2$$

$$\text{where } p_k \rightarrow \text{proportion of samples belonging to class k}$$

$$\text{Therefore, we maximize }\Delta I = I_P - \left(\frac{|S_L|}{|S|}I_L - \frac{|S_R|}{|S|}I_R\right) \text  { at each node}$$

One tiny flaw with this approach is that it is greedy. It finds the best choice according to the local circumstances only and could potentially lead to sub-optimal trees, but the method is usually good enough for most purposes.

One would also stop the growth of a sub-tree if the decrease in impurity isn't significant enough or if the number of samples at a node is smaller than some min_sample number. These measures are also in place to prevent overfitting.

Thus the stopping criterions become:
1) Depth of the tree is too high, or
2) Change in impurity is insignificant or even negative, or
3) Number of samples is too little



In [8]:
class Node():
    def __init__(self, feature_idx=None, threshold=None, left_branch=None, right_branch=None):
        self.feature_idx = feature_idx        
        self.threshold = threshold     
        self.left_branch = left_branch    
        self.right_branch = right_branch 

In [9]:
def isClassification(dtype):
    return dtype in ['int32','int64']

def getSplit(column, threshold):
    left_ids = np.where(column <= threshold)[0]
    right_ids = np.where(column > threshold)[0]
    return left_ids, right_ids

def getImpurity(y):
    if isClassification(y.dtype):
        classes, counts = np.unique(y)
        return 1-np.sum(counts**2)/(len(y)**2)
    else:
        return np.var(y)

def getWeightedImpurity(y, left_ids, right_ids):
    total_len = len(left_ids) + len(right_ids)
    return (len(left_ids)*getImpurity(y[left_ids]) + len(right_ids)*getImpurity(y[right_ids]))/total_len

def getBestSplit(X,y):
    best_left_ids = best_right_ids = best_feature_idx = best_threshold = None
    best_impurity = float("inf") # Gets the largest float value possible
    for feature_idx in range(X.shape[1]):
        thresholds = np.unique(X[:,feature_idx])
        for threshold in thresholds:
            left_ids, right_ids = getSplit(X[:,feature_idx],threshold)
            
            if len(left_ids)*len(right_ids) == 0:
                continue
            
            weighted_impurity = getWeightedImpurity(y, left_ids, right_ids)

            if best_impurity > weighted_impurity:
                best_impurity = weighted_impurity
                best_left_ids = left_ids
                best_right_ids = right_ids
                best_feature_idx = feature_idx
                best_threshold = threshold
                
    return best_feature_idx, best_threshold, best_left_ids, best_right_ids

def mode(vals):
    unique_vals, counts = np.unique(vals,return_countds=True)
    return unique_vals[np.argmax(counts)]

def getDecisionTree(X, y, depth=0, max_depth=None, min_split=2):
    
    if ((max_depth is not None and depth >= max_depth) or getImpurity(y) == 0 or len(y) <= min_split):
        if isClassification(y.dtype): return mode(y)
        else: return np.mean(y)
    
    feature_idx, threshold, left_ids, right_ids = getBestSplit(X,y)

    if feature_idx is None:
        if isClassification(y.dtype): return mode(y)
        else: return np.mean(y)

    left_subtree = getDecisionTree(X[left_ids],y[left_ids], depth + 1, max_depth, min_split)
    right_subtree = getDecisionTree(X[right_ids],y[right_ids], depth + 1, max_depth, min_split)
    
    return Node(feature_idx, threshold, left_subtree, right_subtree)

def getPrediction(tree, x):
    if not isinstance(tree, Node):
        return tree

    feature_idx = tree.feature_idx
    threshold = tree.threshold

    if x[feature_idx] <= threshold:
        return getPrediction(tree.left_branch, x)
    else:
        return getPrediction(tree.right_branch, x)

## Training the decision tree

In [10]:
tree = getDecisionTree(np.array(train_data_X), np.array(train_data_Y), depth = 0, max_depth = 6, min_split = 10)

## Evaluating the tree

In [11]:
predictions = np.array([getPrediction(tree,row) for row in np.array(test_data_X)])
mae = np.sum(np.abs(predictions - np.array(test_data_Y)))/len(test_data_Y)
print(f"Mean absolute error : {round(mae, 2)}")

Mean absolute error : 52488.27
